In [16]:
%matplotlib inline
import pandas as pd

#Importo Decisions.csv
decisions = pd.read_csv("../Dataset 2.0/Decisions.csv")

#Da "Decisions.csv" seleziono solo le tuple riguardanti il round desiderato (1-9)
decisions1 = decisions[decisions["Round"] == 5]

#Ho effettuato un controllo esterno per assicurarmi che tutte le tuple vengano prese in considerazione

#Estraggo "ID", "Session ID", "Worker Agent ID" e "Worker Agent Reputation"
decisions1 = decisions1.iloc[:, [0, 1, 3, 4]]

#Da "Decisions.csv" seleziono solo le tuple riguardanti il round successivo
decisions2 = decisions[decisions["Round"] == 6]

#Estraggo "Session ID", "Worker Agent ID" e "Worker Agent Reputation"
decisions2 = decisions2.iloc[:, [1, 3, 4]]

#Importo TasksQualities per le task assegnate a ciascun WA
queue = pd.read_csv("../Dataset 2.0/TasksQualities.csv", sep = ";")

queue = queue.sort_values(by = "ID")


print("end")

end


In [17]:
from sklearn.linear_model import LinearRegression
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


#Creo un DataFrame vuoto nel quale concatenerò il risultato di ciascuna iterazione che sto per eseguire
totale = pd.DataFrame()

#Creo un DataFrame vuoto nel quale concatenerò i coefficienti di regressione per ogni WA
coeffs = pd.DataFrame()


for i in range(20):
    
    #Faccio coincidere in entrambe le tabelle gli wa scelti (in due round consecutivi)
    tmp1 = decisions1[decisions1["Worker Agent ID"] == i+1]
    tmp2 = decisions2[decisions2["Worker Agent ID"] == i+1]
    tmp1 = pd.merge(tmp1, tmp2, left_on = "Session ID", right_on = "Session ID")
    tmp1 = tmp1.rename(columns = {"Worker Agent ID_x": "Worker Agent ID", "Worker Agent Reputation_x": "Worker Agent Starting Reputation", "Worker Agent Reputation_y": "Worker Agent Observed Reputation"})
    tmp1 = pd.merge(queue, tmp1, left_on = "ID", right_on = "ID")
    tmp1 = tmp1.drop("Worker Agent ID_y", 1)
    tmp1 = tmp1.drop("Session ID", 1)
    tmp1 = tmp1.sort_values(by = "ID")
    
    
    #Preparazione e training del modello per ogni wa riguardo al round scelto
    xtmp = tmp1.iloc[:, 0:93]
    ytmp = tmp1.iloc[:, 93]
    
    reg = LinearRegression().fit(xtmp, ytmp)
    
    
    #Salvataggio dei coefficienti in un Dataframe Unico
    var = pd.DataFrame(reg.coef_, columns = [f"{i + 1}"])

    totale = pd.concat([totale, tmp1])
    
    coeffs[f"{i+1}"] =  var
    coeffs.to_csv("../Coefficienti di Regressione/coeffs.csv", index = False)
    
print("end")

end


In [18]:
totale = totale.sort_values(by = "ID")

#Preparazione del dataset per il training
x = totale.iloc[:, 0:93]
y = totale.iloc[:, 93]
x

,ID,value1,difficulty1,er1,value2,difficulty2,er2,value3,difficulty3,er3,...,difficulty28,er28,value29,difficulty29,er29,value30,difficulty30,er30,Worker Agent ID,Worker Agent Starting Reputation
0,1441,0,0.0,0,0,0.0,0,0,0,0,...,0.8,4,0,0.0,0,0,0.0,0,11,0.201203
0,1443,0,0.0,0,0,0.0,0,5,1,5,...,0.0,0,0,0.0,0,2,0.4,2,16,0.499012
0,1445,0,0.0,0,2,0.4,2,5,1,5,...,0.0,0,3,0.6,3,2,0.4,2,20,0.659975
0,1447,4,0.8,4,0,0.0,0,0,0,0,...,0.8,4,0,0.0,0,0,0.0,0,13,0.346965
0,1449,0,0.0,0,0,0.0,0,0,0,0,...,0.0,0,0,0.0,0,0,0.0,0,14,0.398402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028,1290567,0,0.0,0,0,0.0,0,0,0,0,...,0.0,0,0,0.0,0,2,0.4,2,16,0.545197
1028,1290569,0,0.0,0,0,0.0,0,0,0,0,...,0.0,0,0,0.0,0,0,0.0,0,17,0.769477
1027,1290571,0,0.0,0,0,0.0,0,5,1,5,...,0.0,0,0,0.0,0,0,0.0,0,18,0.554174
1028,1290573,0,0.0,0,0,0.0,0,0,0,0,...,0.0,0,0,0.0,0,0,0.0,0,19,0.799395


In [19]:
#Training del modello utilizzando la Regressione Lineare

reg = LinearRegression().fit(x, y)
reg.score(x, y)

0.8864055596699331

In [20]:
coeffs

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,3.811742e-09,6.048641e-09,-4.709667e-09,-1.827285e-08,7.806680e-10,3.789982e-09,-1.109386e-08,-1.382319e-09,9.171717e-10,-4.438080e-09,2.699228e-10,-2.410377e-09,5.693969e-09,-1.279506e-08,-7.512034e-09,5.289049e-09,-6.926784e-09,-2.172075e-09,2.774423e-09,8.525997e-09
1,6.749094e-04,-7.006359e-04,5.090151e-05,-2.331680e-03,-3.427420e-03,-3.131193e-03,-2.191040e-03,-1.557389e-03,-1.787463e-03,-1.804293e-03,-1.307998e-03,-1.792173e-03,-2.291123e-03,-1.058651e-03,-1.289142e-03,-1.204137e-03,-2.103278e-03,-7.093477e-04,-1.892691e-03,-9.021352e-04
2,1.349819e-04,-1.401272e-04,1.018030e-05,-4.663359e-04,-6.854840e-04,-6.262386e-04,-4.382080e-04,-3.114778e-04,-3.574925e-04,-3.608586e-04,-2.615997e-04,-3.584347e-04,-4.582247e-04,-2.117302e-04,-2.578283e-04,-2.408274e-04,-4.206557e-04,-1.418695e-04,-3.785381e-04,-1.804270e-04
3,6.749094e-04,-7.006359e-04,5.090151e-05,-2.331680e-03,-3.427420e-03,-3.131193e-03,-2.191040e-03,-1.557389e-03,-1.787463e-03,-1.804293e-03,-1.307998e-03,-1.792173e-03,-2.291123e-03,-1.058651e-03,-1.289142e-03,-1.204137e-03,-2.103278e-03,-7.093477e-04,-1.892691e-03,-9.021352e-04
4,-4.029438e-03,-3.463759e-03,2.862049e-03,1.270166e-03,2.624775e-03,1.872885e-03,-3.885712e-03,-1.058018e-03,-3.276103e-03,-4.083538e-03,-4.846233e-03,5.025610e-04,1.592788e-03,1.967582e-03,-4.016335e-04,-1.919236e-03,-1.807108e-03,-5.027133e-04,2.484486e-04,-1.918691e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,-2.915283e-03,3.189024e-04,-8.918850e-04,3.294386e-03,-1.659945e-03,-1.655610e-03,-1.113590e-03,-1.115601e-03,-4.309405e-03,3.447233e-04,-8.758846e-04,3.599856e-05,5.688276e-04,1.813234e-03,-2.729386e-03,-1.583999e-03,-1.153534e-03,9.445861e-04,6.201213e-04,-5.605632e-04
89,-5.830566e-04,6.378049e-05,-1.783770e-04,6.588772e-04,-3.319889e-04,-3.311221e-04,-2.227180e-04,-2.231202e-04,-8.618809e-04,6.894466e-05,-1.751769e-04,7.199712e-06,1.137655e-04,3.626468e-04,-5.458773e-04,-3.167998e-04,-2.307069e-04,1.889172e-04,1.240243e-04,-1.121126e-04
90,-2.915283e-03,3.189024e-04,-8.918850e-04,3.294386e-03,-1.659945e-03,-1.655610e-03,-1.113590e-03,-1.115601e-03,-4.309405e-03,3.447233e-04,-8.758846e-04,3.599856e-05,5.688276e-04,1.813234e-03,-2.729386e-03,-1.583999e-03,-1.153534e-03,9.445861e-04,6.201213e-04,-5.605632e-04
91,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [40]:
#Training del modello utilizzando SVR

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(x, y)
regr.score(x, y)

0.788342762977449